In [2]:
import os

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\Meet\\OneDrive\\Desktop\\engg\\MLOps\\Projects\\Project Alpha\\Project-Alpha'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [8]:
from src.projectAlpha.constants import *
from src.projectAlpha.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH,schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        return data_ingestion_config

In [12]:
import os
import urllib.request as request
from src.projectAlpha import logger
import zipfile

In [13]:
## component- data ingestion
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url= self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info \n{headers}")
        else:
            logger.info("file already exists")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-09 13:43:11,791: INFO: common: yaml file <_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='UTF-8'> loaded successfully]
[2024-12-09 13:43:11,793: INFO: common: yaml file <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully]
[2024-12-09 13:43:11,798: INFO: common: yaml file <_io.TextIOWrapper name='schema.yaml' mode='r' encoding='UTF-8'> loaded successfully]
[2024-12-09 13:43:11,801: INFO: common: created directory as artifacts]
[2024-12-09 13:43:11,803: INFO: common: created directory as artifacts/data_ingestion]


[2024-12-09 13:43:23,777: INFO: 1789186643: artifacts/data_ingestion/data.zip downloaded! with following info 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 0D79:F2829:217565:2CC771:6756A6A3
Accept-Ranges: bytes
Date: Mon, 09 Dec 2024 08:13:23 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100033-HYD
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1733732004.639543,VS0,VE359
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: f7516dc9d65596d2e6a30fb97af0188f7d80abb1
Expires: Mon, 09 Dec 2024 08:18:23 GMT
Source-Age: 0

]
